In [1]:
import os
import time
import numpy as np
import tensorflow as tf

from src.datasets import load_dataset, preprocess_dataset, prefetch_dataset
from src.pmi_estimators import train_critic_model, neural_pmi
from src.utils import *

2024-02-01 07:51:52.172700: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 07:51:52.172764: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 07:51:52.172788: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 07:51:52.179889: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
cfg = { 'dataset' : 'mnist',
        'model' : 'mlp',
        'batch_size' : 512,
        'optimizer' : 'Adam',
        'learning_rate' : 0.001,
        'max_epoch' : 300,
        'patience' : 10,}    

model_name = cfg['model']
dataset_name = cfg['dataset']

In [3]:
##############################################################
#
# Compute classification error
#
# #############################################################
ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
n_classes = ds_info.features['label'].num_classes
ds_train = preprocess_dataset(ds_train, cfg, n_classes, resize=False, normalize=True, onehot=True)
ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=False, normalize=True, onehot=True)
ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=False, normalize=True, onehot=True)
ds_train = prefetch_dataset(ds_train, batch_size=cfg['batch_size'])
ds_val = prefetch_dataset(ds_val, batch_size=cfg['batch_size'])
ds_test = prefetch_dataset(ds_test, batch_size=cfg['batch_size'])

train_acc = []
val_acc = []
test_acc = []
for run in range(5):
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
    train_acc.append(model.evaluate(ds_train, verbose=0)[1])
    val_acc.append(model.evaluate(ds_val, verbose=0)[1])
    test_acc.append(model.evaluate(ds_test, verbose=0)[1])
print(f'Average train error: {(100-np.mean(train_acc)*100):.2f}, std: {(np.std(train_acc)*100):.2f}')
print(f'Average validation error: {(100-np.mean(val_acc)*100):.2f}, std: {(np.std(val_acc)*100):.2f}')
print(f'Average test error: {(100-np.mean(test_acc)*100):.2f}, std: {(np.std(test_acc)*100):.2f}')

2024-01-30 16:59:43.603447: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1128 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:47:00.0, compute capability: 8.0
2024-01-30 16:59:44.937705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8906


Average train error: 4.78, std: 1.11
Average validation error: 19.40, std: 0.38
Average test error: 20.17, std: 0.39


In [4]:
critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']
epochs = 100

for run in range(5):
    print(f'Run: {run+1}')
    for critic in critic_list:
        for estimator in estimators_list:
            tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'
            if not os.path.exists(exp_name):
                print("Making directory", exp_name)
                os.makedirs(exp_name)
                
            ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
            n_classes = ds_info.features['label'].num_classes
            ds_train = preprocess_dataset(ds_train, cfg, n_classes, resize=False, normalize=True, onehot=True)
            ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=False, normalize=True, onehot=True)
            ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=False, normalize=True, onehot=True)
            
            model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
            int_model = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)

            ##############################################################
            #
            # Train PMI Model
            #
            # #############################################################
        
            print(f'Training PMI model ({critic}, {estimator})...')
            ds_activity = ds_train.batch(cfg['batch_size']).map(lambda x, y: (int_model(x), y)).cache().prefetch(tf.data.AUTOTUNE)
            train_critic_model(ds_activity, critic=critic, estimator=estimator, epochs=epochs, save_path=f'{exp_name}/pmi_model')
                                                                              
            ##############################################################
            #
            # Compute PMI for all validation and test samples
            #
            # #############################################################

            pmi_model = tf.keras.models.load_model(f'{exp_name}/pmi_model')
            
            print(f'Computing PMI for all validation samples and for all classes...')
            pmi_class = []
            for k in range(n_classes):
                ds_activity = ds_val.batch(cfg['batch_size']).map(lambda x, y: (int_model(x), tf.one_hot(tf.fill([tf.shape(x)[0]], k), depth=n_classes))).cache().prefetch(tf.data.AUTOTUNE)
                pmi_list = []
                for (x_batch, y_batch) in ds_activity:
                    pmi = neural_pmi(x_batch, y_batch, pmi_model, estimator=estimator)
                    pmi_list += np.array(pmi).tolist()
                pmi_class.append(pmi_list)
            np.save(f'{exp_name}/pmi_class_val.npy', np.array(pmi_class).T)
            
            print(f'Computing PMI for all test samples and for all classes...')
            pmi_class = []
            for k in range(n_classes):
                ds_activity = ds_test.batch(cfg['batch_size']).map(lambda x, y: (int_model(x), tf.one_hot(tf.fill([tf.shape(x)[0]], k), depth=n_classes))).cache().prefetch(tf.data.AUTOTUNE)
                pmi_list = []
                for (x_batch, y_batch) in ds_activity:
                    pmi = neural_pmi(x_batch, y_batch, pmi_model, estimator=estimator)
                    pmi_list += np.array(pmi).tolist()
                pmi_class.append(pmi_list)
            np.save(f'{exp_name}/pmi_class_test.npy', np.array(pmi_class).T)

Run: 1
Making directory ../results/PI_Explainability/cnn_cifar10/run_1/calibration/pmi/concat_probabilistic_classifier
Training PMI model (concat, probabilistic_classifier)...


2024-01-30 13:22:49.210875: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f57f2346c80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-30 13:22:49.210921: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-01-30 13:22:49.215881: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-30 13:22:49.337300: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_1/calibration/pmi/concat_probabilistic_classifier/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_1/calibration/pmi/concat_probabilistic_classifier/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Making directory ../results/PI_Explainability/cnn_cifar10/run_1/calibration/pmi/concat_density_ratio_fitting
Training PMI model (concat, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_1/calibration/pmi/concat_density_ratio_fitting/pmi_model/assets


...

Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Making directory ../results/PI_Explainability/cnn_cifar10/run_5/calibration/pmi/separable_density_ratio_fitting
Training PMI model (separable, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_5/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_5/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Making directory ../results/PI_Explainability/cnn_cifar10/run_5/calibration/pmi/separable_variational_f_js
Training PMI model (separable, variational_f_js)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_5/calibration/pmi/separable_variational_f_js/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/cnn_cifar10/run_5/calibration/pmi/separable_variational_f_js/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...


In [5]:
##############################################################
#
# Compute Filtering Accuracy (without softmax scaling)
#
# #############################################################

critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']

for run in range(5):
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    print(f'Run: {run+1}')
    for critic in critic_list:
        for estimator in estimators_list:
            print(f'Critic: {critic}, Estimator: {estimator}')
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'

            ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
            n_classes = ds_info.features['label'].num_classes
            ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=False, normalize=True, onehot=True)
            ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=False, normalize=True, onehot=True)
            model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
            
            true_y = np.argmax([y for x,y in ds_val], axis=1)
            pred_y = np.argmax(model.predict(ds_val.batch(cfg['batch_size']), verbose=0), axis=1)
            true_label = np.equal(true_y, pred_y).astype(int) # assign 0 if true_y != pred_y, assign 1 if true_y == pred_y
            pmi_class = np.load(f'{exp_name}/pmi_class_val.npy')
            pmi = [pmi_value[pred_value] for pmi_value, pred_value in zip(pmi_class, pred_y)]
            opt_threshold = compute_opt_threshold(pmi, true_label)
        
            true_y = np.argmax([y for x,y in ds_test], axis=1)
            pred_y = np.argmax(model.predict(ds_test.batch(cfg['batch_size']), verbose=0), axis=1)
            true_label = np.equal(true_y, pred_y).astype(int) # assign 0 if true_y != pred_y, assign 1 if true_y == pred_y
            pmi_class = np.load(f'{exp_name}/pmi_class_test.npy')
            pmi = [pmi_value[pred_value] for pmi_value, pred_value in zip(pmi_class, pred_y)]
            test_filtering_acc = compute_filtering_acc(pmi, true_label, opt_threshold)
            
            np.savez(f'{exp_name}/unscaled_filtering_accuracy.npz', opt_threshold=opt_threshold, test_filtering_acc=test_filtering_acc)
            print(f'Opt. threshold: {opt_threshold:.3f}, Test filtering error:{100-test_filtering_acc:.2f}')

Run: 1
Critic: concat, Estimator: probabilistic_classifier
Opt. threshold: -3.779, Test filtering error:-79.05
Critic: concat, Estimator: density_ratio_fitting
Opt. threshold: 2.452, Test filtering error:-81.69
Critic: concat, Estimator: variational_f_js
Opt. threshold: -1.094, Test filtering error:-79.99
Critic: separable, Estimator: probabilistic_classifier
Opt. threshold: 1.234, Test filtering error:-79.90
Critic: separable, Estimator: density_ratio_fitting
Opt. threshold: 1.848, Test filtering error:-80.90
Critic: separable, Estimator: variational_f_js
Opt. threshold: -2.010, Test filtering error:-79.81
Run: 2
Critic: concat, Estimator: probabilistic_classifier
Opt. threshold: -7.448, Test filtering error:-78.93
Critic: concat, Estimator: density_ratio_fitting
Opt. threshold: 2.194, Test filtering error:-82.36
Critic: concat, Estimator: variational_f_js
Opt. threshold: 0.140, Test filtering error:-81.36
Critic: separable, Estimator: probabilistic_classifier
Opt. threshold: 1.521, T

In [4]:
critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']

for critic in critic_list:
    for estimator in estimators_list:
        threshold = []
        filtering_acc = []
        for run in range(5):
            tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'
            f = np.load(f'{exp_name}/unscaled_filtering_accuracy.npz')
            opt_threshold = f['opt_threshold']
            test_filtering_acc = f['test_filtering_acc']
            threshold.append(opt_threshold)
            filtering_acc.append(test_filtering_acc)
            
        print('-----------------------------')
        print(f'Critic: {critic}, Estimator: {estimator}')
        print(f'Average opt. threshold: {(np.mean(threshold)):.3f}, std: {(np.std(threshold)):.3f}')
        print(f'Average test filtering error: {(100-np.mean(filtering_acc)):.2f}, std: {(np.std(filtering_acc)):.2f}')

-----------------------------
Critic: concat, Estimator: probabilistic_classifier
Average opt. threshold: -6.336, std: 2.255
Average test filtering error: 20.18, std: 0.38
-----------------------------
Critic: concat, Estimator: density_ratio_fitting
Average opt. threshold: 2.369, std: 0.090
Average test filtering error: 17.23, std: 0.48
-----------------------------
Critic: concat, Estimator: variational_f_js
Average opt. threshold: 0.071, std: 0.624
Average test filtering error: 18.41, std: 0.64
-----------------------------
Critic: separable, Estimator: probabilistic_classifier
Average opt. threshold: 1.226, std: 0.290
Average test filtering error: 18.84, std: 0.50
-----------------------------
Critic: separable, Estimator: density_ratio_fitting
Average opt. threshold: 2.035, std: 0.204
Average test filtering error: 17.79, std: 0.51
-----------------------------
Critic: separable, Estimator: variational_f_js
Average opt. threshold: -0.634, std: 0.762
Average test filtering error: 19

In [3]:
##############################################################
#
# Compute Filtering Accuracy (with softmax scaling)
#
# #############################################################

critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']

for run in range(5):
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    print(f'Run: {run+1}')
    for critic in critic_list:
        for estimator in estimators_list:
            print(f'Critic: {critic}, Estimator: {estimator}')
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'

            ds_train, ds_val, ds_test, ds_info = load_dataset(cfg, shuffle=False)
            n_classes = ds_info.features['label'].num_classes
            ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=False, normalize=True, onehot=True)
            ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=False, normalize=True, onehot=True)
            model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
            
            true_y = np.argmax([y for x,y in ds_val], axis=1)
            pred_y = np.argmax(model.predict(ds_val.batch(cfg['batch_size']), verbose=0), axis=1)
            true_label = np.equal(true_y, pred_y).astype(int) # assign 0 if true_y != pred_y, assign 1 if true_y == pred_y
            pmi_class = np.load(f'{exp_name}/pmi_class_val.npy')
            pmi_class = np.array([softmax(x) for x in pmi_class])
            pmi = [pmi_value[pred_value] for pmi_value, pred_value in zip(pmi_class, pred_y)]
            opt_threshold = compute_opt_threshold(pmi, true_label)
        
            true_y = np.argmax([y for x,y in ds_test], axis=1)
            pred_y = np.argmax(model.predict(ds_test.batch(cfg['batch_size']), verbose=0), axis=1)
            true_label = np.equal(true_y, pred_y).astype(int) # assign 0 if true_y != pred_y, assign 1 if true_y == pred_y
            pmi_class = np.load(f'{exp_name}/pmi_class_test.npy')
            pmi_class = np.array([softmax(x) for x in pmi_class])
            pmi = [pmi_value[pred_value] for pmi_value, pred_value in zip(pmi_class, pred_y)]
            test_filtering_acc = compute_filtering_acc(pmi, true_label, opt_threshold)
            
            np.savez(f'{exp_name}/scaled_filtering_accuracy.npz', opt_threshold=opt_threshold, test_filtering_acc=test_filtering_acc)
            print(f'Opt. threshold: {opt_threshold:.3f}, Test filtering error:{100-test_filtering_acc:.2f}')

Run: 1
Critic: concat, Estimator: probabilistic_classifier


2024-02-01 07:53:26.740701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1883] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78835 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


Opt. threshold: 0.531, Test filtering error:1.71
Critic: concat, Estimator: density_ratio_fitting
Opt. threshold: 0.407, Test filtering error:1.64
Critic: concat, Estimator: variational_f_js
Opt. threshold: 0.646, Test filtering error:1.65
Critic: separable, Estimator: probabilistic_classifier
Opt. threshold: 0.419, Test filtering error:1.63
Critic: separable, Estimator: density_ratio_fitting
Opt. threshold: 0.399, Test filtering error:1.71
Critic: separable, Estimator: variational_f_js
Opt. threshold: 0.032, Test filtering error:1.73
Run: 2
Critic: concat, Estimator: probabilistic_classifier
Opt. threshold: 0.000, Test filtering error:1.90
Critic: concat, Estimator: density_ratio_fitting
Opt. threshold: 0.173, Test filtering error:1.84
Critic: concat, Estimator: variational_f_js
Opt. threshold: 0.147, Test filtering error:1.79
Critic: separable, Estimator: probabilistic_classifier
Opt. threshold: 0.141, Test filtering error:1.73
Critic: separable, Estimator: density_ratio_fitting
Opt.

In [5]:
critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']

for critic in critic_list:
    for estimator in estimators_list:
        threshold = []
        filtering_acc = []
        for run in range(5):
            tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'
            f = np.load(f'{exp_name}/scaled_filtering_accuracy.npz')
            opt_threshold = f['opt_threshold']
            test_filtering_acc = f['test_filtering_acc']
            threshold.append(opt_threshold)
            filtering_acc.append(test_filtering_acc)
            
        print('-----------------------------')
        print(f'Critic: {critic}, Estimator: {estimator}')
        print(f'Average opt. threshold: {(np.mean(threshold)):.3f}, std: {(np.std(threshold)):.3f}')
        print(f'Average test filtering error: {(100-np.mean(filtering_acc)):.2f}, std: {(np.std(filtering_acc)):.2f}')

-----------------------------
Critic: concat, Estimator: probabilistic_classifier
Average opt. threshold: 0.394, std: 0.255
Average test filtering error: 1.78, std: 0.08
-----------------------------
Critic: concat, Estimator: density_ratio_fitting
Average opt. threshold: 0.405, std: 0.125
Average test filtering error: 1.68, std: 0.08
-----------------------------
Critic: concat, Estimator: variational_f_js
Average opt. threshold: 0.452, std: 0.166
Average test filtering error: 1.69, std: 0.05
-----------------------------
Critic: separable, Estimator: probabilistic_classifier
Average opt. threshold: 0.479, std: 0.205
Average test filtering error: 1.69, std: 0.04
-----------------------------
Critic: separable, Estimator: density_ratio_fitting
Average opt. threshold: 0.444, std: 0.076
Average test filtering error: 1.74, std: 0.04
-----------------------------
Critic: separable, Estimator: variational_f_js
Average opt. threshold: 0.225, std: 0.204
Average test filtering error: 1.70, std

In [22]:
##############################################################
#
# Compute ECE (with softmax scaling)
#
# #############################################################

critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']

for run in range(5):
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    print(f'Run: {run+1}')
    for critic in critic_list:
        for estimator in estimators_list:
            print(f'Critic: {critic}, Estimator: {estimator}')
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'

            ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
            n_classes = ds_info.features['label'].num_classes
            ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=False, normalize=True, onehot=True)
            model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
            
            true_y = np.argmax([y for x,y in ds_test], axis=1)
            pred_y = np.argmax(model.predict(ds_test.batch(cfg['batch_size']), verbose=0), axis=1)
            pmi_class = np.load(f'{exp_name}/pmi_class_test.npy')
            pmi_class = np.array([softmax(x) for x in pmi_class])
            pmi = np.array([pmi_value[pred_value] for pmi_value, pred_value in zip(pmi_class, pred_y)])
            ece = compute_ece(pmi, true_y, pred_y, n_bins=10)
            
            np.save(f'{exp_name}/ece_test.npy', ece)
            print(f'ECE: {ece:.2f}')

Run: 1
Critic: concat, Estimator: probabilistic_classifier
ECE: 1.08
Critic: concat, Estimator: density_ratio_fitting
ECE: 2.01
Critic: concat, Estimator: variational_f_js
ECE: 1.26
Critic: separable, Estimator: probabilistic_classifier
ECE: 1.17
Critic: separable, Estimator: density_ratio_fitting
ECE: 0.93
Critic: separable, Estimator: variational_f_js
ECE: 1.41
Run: 2
Critic: concat, Estimator: probabilistic_classifier
ECE: 0.99
Critic: concat, Estimator: density_ratio_fitting
ECE: 2.53
Critic: concat, Estimator: variational_f_js
ECE: 1.29
Critic: separable, Estimator: probabilistic_classifier
ECE: 1.13
Critic: separable, Estimator: density_ratio_fitting
ECE: 0.75
Critic: separable, Estimator: variational_f_js
ECE: 1.37
Run: 3
Critic: concat, Estimator: probabilistic_classifier
ECE: 1.05
Critic: concat, Estimator: density_ratio_fitting
ECE: 4.05
Critic: concat, Estimator: variational_f_js
ECE: 1.34
Critic: separable, Estimator: probabilistic_classifier
ECE: 1.09
Critic: separable, Es

In [23]:
critic_list = ['concat', 'separable']
estimators_list = ['probabilistic_classifier', 'density_ratio_fitting', 'variational_f_js']

for critic in critic_list:
    for estimator in estimators_list:
        ece_list = []
        for run in range(5):
            tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
            exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'
            ece = np.load(f'{exp_name}/ece_test.npy')
            ece_list.append(ece)
            
        print('-----------------------------')
        print(f'Critic: {critic}, Estimator: {estimator}')
        print(f'Average ECE: {(np.mean(ece_list)):.2f}, std: {(np.std(ece_list)):.2f}')

-----------------------------
Critic: concat, Estimator: probabilistic_classifier
Average ECE: 1.08, std: 0.08
-----------------------------
Critic: concat, Estimator: density_ratio_fitting
Average ECE: 2.82, std: 0.80
-----------------------------
Critic: concat, Estimator: variational_f_js
Average ECE: 1.33, std: 0.05
-----------------------------
Critic: separable, Estimator: probabilistic_classifier
Average ECE: 1.17, std: 0.07
-----------------------------
Critic: separable, Estimator: density_ratio_fitting
Average ECE: 0.83, std: 0.08
-----------------------------
Critic: separable, Estimator: variational_f_js
Average ECE: 1.42, std: 0.05


In [4]:
cfg = { 'dataset' : 'stl10',
        'model' : 'pretrained_inception',
        'batch_size' : 512,
        'optimizer' : 'Adam',
        'learning_rate' : 0.001,
        'max_epoch' : 300,
        'patience' : 10,}    

model_name = cfg['model']
dataset_name = cfg['dataset']

In [6]:
critic = 'separable'
estimator = 'density_ratio_fitting'
epochs = 100

for run in range(5):
    print(f'Run: {run+1}')
    tf.keras.utils.set_random_seed(run+10) # set random seed for Python, NumPy, and TensorFlow
    exp_name = f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/calibration/pmi/{critic}_{estimator}'
    if not os.path.exists(exp_name):
        print("Making directory", exp_name)
        os.makedirs(exp_name)

    ds_train, ds_val, ds_test, ds_info = load_dataset(cfg)
    n_classes = ds_info.features['label'].num_classes
    ds_train = preprocess_dataset(ds_train, cfg, n_classes, resize=True, normalize=True, onehot=True)
    ds_val = preprocess_dataset(ds_val, cfg, n_classes, resize=True, normalize=True, onehot=True)
    ds_test = preprocess_dataset(ds_test, cfg, n_classes, resize=True, normalize=True, onehot=True)

    model = tf.keras.models.load_model(f'../results/PI_Explainability/{model_name}_{dataset_name}/run_{run+1}/trained_model.keras')
    int_model = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)

    ##############################################################
    #
    # Train PMI Model
    #
    # #############################################################

    print(f'Training PMI model ({critic}, {estimator})...')
    ds_activity = ds_train.batch(cfg['batch_size']).map(lambda x, y: (int_model(x), y)).cache().prefetch(tf.data.AUTOTUNE)
    train_critic_model(ds_activity, critic=critic, estimator=estimator, epochs=epochs, save_path=f'{exp_name}/pmi_model')

    ##############################################################
    #
    # Compute PMI for all validation and test samples
    #
    # #############################################################

    pmi_model = tf.keras.models.load_model(f'{exp_name}/pmi_model')

    print(f'Computing PMI for all validation samples and for all classes...')
    pmi_class = []
    for k in range(n_classes):
        ds_activity = ds_val.batch(cfg['batch_size']).map(lambda x, y: (int_model(x), tf.one_hot(tf.fill([tf.shape(x)[0]], k), depth=n_classes))).cache().prefetch(tf.data.AUTOTUNE)
        pmi_list = []
        for (x_batch, y_batch) in ds_activity:
            pmi = neural_pmi(x_batch, y_batch, pmi_model, estimator=estimator)
            pmi_list += np.array(pmi).tolist()
        pmi_class.append(pmi_list)
    np.save(f'{exp_name}/pmi_class_val.npy', np.array(pmi_class).T)

    print(f'Computing PMI for all test samples and for all classes...')
    pmi_class = []
    for k in range(n_classes):
        ds_activity = ds_test.batch(cfg['batch_size']).map(lambda x, y: (int_model(x), tf.one_hot(tf.fill([tf.shape(x)[0]], k), depth=n_classes))).cache().prefetch(tf.data.AUTOTUNE)
        pmi_list = []
        for (x_batch, y_batch) in ds_activity:
            pmi = neural_pmi(x_batch, y_batch, pmi_model, estimator=estimator)
            pmi_list += np.array(pmi).tolist()
        pmi_class.append(pmi_list)
    np.save(f'{exp_name}/pmi_class_test.npy', np.array(pmi_class).T)

Run: 1
Training PMI model (separable, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_1/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_1/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Run: 2
Making directory ../results/PI_Explainability/pretrained_inception_stl10/run_2/calibration/pmi/separable_density_ratio_fitting
Training PMI model (separable, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_2/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_2/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Run: 3
Making directory ../results/PI_Explainability/pretrained_inception_stl10/run_3/calibration/pmi/separable_density_ratio_fitting
Training PMI model (separable, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_3/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


...

Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Run: 4
Making directory ../results/PI_Explainability/pretrained_inception_stl10/run_4/calibration/pmi/separable_density_ratio_fitting
Training PMI model (separable, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_4/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_4/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
Run: 5
Making directory ../results/PI_Explainability/pretrained_inception_stl10/run_5/calibration/pmi/separable_density_ratio_fitting
Training PMI model (separable, density_ratio_fitting)...
INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_5/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


INFO:tensorflow:Assets written to: ../results/PI_Explainability/pretrained_inception_stl10/run_5/calibration/pmi/separable_density_ratio_fitting/pmi_model/assets


Computing PMI for all validation samples and for all classes...
Computing PMI for all test samples and for all classes...
